In [8]:
import nltk
nltk.download('stopwords')
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive

drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
def cross_validation(A, b):
    A = np.array(A)
    b = np.array(b)
    k = 5
    fold_size = len(A) // k
    accu= []
    for i in range(k):
        start = i * fold_size
        end = (i + 1) * fold_size if i < k - 1 else len(A)
        A_train = np.concatenate([A[:start], A[end:]])
        b_train = np.concatenate([b[:start], b[end:]])
        A_val = A[start:end]
        b_val = b[start:end]
        vector = TfidfVectorizer(norm='l2', min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, ngram_range=(1, 2), max_features=9000)
        train_vector = vector.fit_transform(A_train)
        test_vector = vector.transform(A_val)
        cosine_ = cosine_similarity(test_vector, train_vector)
        result = Knn(cosine_, 500)
        accuracy = sum(1 for pred, true in zip(result, b_val) if pred == true) / len(b_val)
        accu.append(accuracy)
    return np.mean(accuracy)


In [13]:

def Knn(Sm, Knn):
    nb = []
    for row in Sm:
        nindices = np.argsort(-row)[:Knn]
        lables = [trainl[idx] for idx in nindices]
        pos_ = sum(1 for label in lables if label == "+1")
        cn = sum(1 for label in lables if label == "-1")
        n = "+1" if pos_ > cn else "-1"
        nb.append(n)
    return nb


In [11]:
def pre(text):
    text = BeautifulSoup(text, "lxml").get_text()
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    words = text.split()
    stop__words = set(stopwords.words("english"))
    word__ = [word for word in words if word not in stop__words]
    return " ".join(word__)

In [15]:

train_data = open('/content/drive/MyDrive/Deepika/train_new.txt', "r")
test_data = open('/content/drive/MyDrive/Deepika/test_new.txt', "r")
train_ = train_data.readlines()
trainl = [line[:2] for line in train_]
train__ = [pre(line[2:]) for line in train_]
testl = test_data.readlines()
testr = [pre(line) for line in testl]
accuracy = cross_validation(train__, trainl)
TFIDF = TfidfVectorizer(norm='l2', min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, ngram_range=(1, 2), max_features=9000)
train_v = TFIDF.fit_transform(train__)
test_v = TFIDF.transform(testr)
simila = cosine_similarity(test_v, train_v)
result = Knn(simila, 500)
print("Average accuracy:", accuracy)
with open('/content/drive/MyDrive/Deepika/output.txt', 'w') as file:
    for r in result:
        file.write(r + '\n')


<ipython-input-11-ce04338e91fd>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "lxml").get_text()


Average accuracy: 0.841
